In [15]:
# imports
import pandas as pd
import numpy as np
import os
from importlib import reload
import stage
import stageConstants
from datetime import datetime, timedelta

In [16]:
CREATE_DATA = False
# the first stage of the race
RACE = 'tour-de-france'
YEAR = 2020
STAGE = 1
URL1 = 'https://www.procyclingstats.com/race/{}/{}/stage-{}/result/result'.format(RACE, YEAR, STAGE)

folder_loc = './temp_data/{}'.format(RACE)
if not os.path.exists(folder_loc):
    os.mkdir(folder_loc)
folder_loc = '{}/{}'.format(folder_loc, YEAR)
if not os.path.exists(folder_loc):
    os.mkdir(folder_loc)
    
# saved data locations
RIDERS_LOC = '{}/riders_df.csv'.format(folder_loc)
RACES_LOC = '{}/races_df.csv'.format(folder_loc)

CLEANED_RACE_HIST = '{}/cleaned_races_history.csv'.format(folder_loc)
CLEANED_TEAMS = '{}/cleaned_teams.csv'.format(folder_loc)
CLEANED_STAGES = '{}/cleaned_stages.csv'.format(folder_loc)

def reload_libraries():
    reload(stageConstants)
    reload(stage)
    print('Reloaded')

In [17]:
riders_df = pd.read_csv(RIDERS_LOC, index_col='riderID')
races_df = pd.read_csv(RACES_LOC, index_col='raceID')
races_hist = pd.read_csv(CLEANED_RACE_HIST, index_col='Unnamed: 0')
teams_hist_df = pd.read_csv(CLEANED_TEAMS, index_col='Unnamed: 0')
stages_df = pd.read_csv(CLEANED_STAGES, index_col='stageID')

In [18]:
riders_df.head()

,name,age,url,team
riderID,,,,
0,ALAPHILIPPE JULIAN,28,rider/julian-alaphilippe,DECEUNINCK - QUICK STEP
1,AMADOR ANDREY,34,rider/andrey-amador,INEOS GRENADIERS
2,ANACONA WINNER,32,rider/winner-anacona,TEAM ARKÉA SAMSIC
3,ARNDT NIKIAS,28,rider/nikias-arndt,TEAM SUNWEB
4,ARU FABIO,30,rider/fabio-aru,UAE-TEAM EMIRATES


In [19]:
races_df.head()

,race,url,cutUrl,year
raceID,,,,
0,Critérium du Dauphiné,race/dauphine/2020/stage-1,race/dauphine/2020,2020
1,Czech Tour,race/czech-cycling-tour/2020/stage-1,race/czech-cycling-tour/2020,2020
2,Etoile de Bessèges,race/etoile-de-besseges/2020/stage-1,race/etoile-de-besseges/2020,2020
3,Jayco Herald Sun Tour,race/herald-sun-tour/2020/stage-1,race/herald-sun-tour/2020,2020
4,La Route d'Occitanie - La Dépêche du Midi,race/la-route-d-occitanie/2020/stage-1,race/la-route-d-occitanie/2020,2020


In [20]:
teams_hist_df.head()

,teamPos,teamName,teamTime,stageID,time
0,1,Team Arkéa Samsic,52:07:18,141,2 days 04:07:18.000000000
1,2,Groupama - FDJ,00:01:01,141,2 days 04:08:19.000000000
2,3,Team Sunweb,00:03:52,141,2 days 04:11:10.000000000
3,4,Caja Rural - Seguros RGA,00:05:00,141,2 days 04:12:18.000000000
4,5,"Cofidis, Solutions Crédits",00:07:41,141,2 days 04:14:59.000000000


In [21]:
stages_df.head()

,stage,url,distance,raceID,stageType,dateFull,averageSpeed,parcoursType,PCSPointScale,start,end,mountains,numMount,racePoints,scored
stageID,,,,,,,,,,,,,,,
0,Bretagne Classic - Ouest-France,race/bretagne-classic/2020/result,247.75,27,0,25th August 2020,41.15,45,1.WT.B,Plouay,Plouay,"['Restergal', 'Côte du Lezot', 'Restergal', 'C...",5,390,True
1,Cadel Evans Great Ocean Road Race,race/great-ocean-race/2020/result,171.10,28,0,2nd February 2020,41.76,53,1.WT.C,Geelong,Geelong,[],0,395,True
2,Circuito de Getxo-Memorial Hermanos Otxoa,race/circuito-de-getxo/2020/result,177.00,29,0,2nd August 2020,42.29,0,1.1,Getxo,Getxo,[],0,238,False
3,Clasica de Almeria,race/clasica-de-almeria/2020/result,187.60,30,0,16th February 2020,42.63,0,1.HC,Roquetas de Mar,Roquetas de Mar,[],0,397,False
4,European Continental Championships - ITT,race/uec-road-european-championships-itt/2020/...,25.60,31,6,24th August 2020,50.69,0,UCI.Cont.Ch.TT,Plouay,Plouay,[],0,86,False


In [28]:
relevant_riders_history = races_hist[races_hist.name.str.upper().isin(riders_df.name.unique())]
for riderID, rider in riders_df.iterrows():
    relevant_riders_history.loc[relevant_riders_history.name.str.upper() == rider['name'], 'riderID'] = riderID

relevant_riders_history.drop(['url', 'name'], axis=1, inplace=True)
relevant_riders_history = relevant_riders_history.astype({'riderID': int})
relevant_riders_history.iloc[:10, :16]

,bib,age,createBib,gcPos,gcTotalTime,greenPnts,greenPos,komPnts,komPos,pnt,stageID,stagePos,stageTime,teamName,uciGc,uciStage
1,122,32.0,False,2,00:00:01,0,117,0,46,150,0,2,0:01,Mitchelton-Scott,0,320
75,111,25.0,False,76,00:06:40,0,117,0,46,5,0,76,6:40,Deceuninck - Quick Step,0,0
121,53,27.0,False,105,00:14:06,0,117,0,46,0,0,105,14:06,CCC Team,0,0
145,186,27.0,False,105,00:14:06,0,117,0,46,0,0,105,14:06,Team Arkéa Samsic,0,0
159,15,36.0,False,1,4:05:49,0,117,0,46,200,1,1,4:05:49,Deceuninck - Quick Step,0,300
160,34,22.0,False,2,4:05:49,0,117,0,46,150,1,2,4:05:49,Team INEOS,0,250
161,51,35.0,False,3,00:00:04,0,117,0,46,120,1,3,0:04,Mitchelton-Scott,0,215
162,63,31.0,False,4,00:00:04,0,117,0,46,100,1,4,0:04,EF Pro Cycling,0,175
163,36,27.0,False,5,00:00:04,0,117,0,46,90,1,5,0:04,Team INEOS,0,120
165,41,25.0,False,7,00:00:25,0,117,0,46,70,1,7,0:25,Lotto Soudal,0,95


In [27]:
relevant_riders_history.iloc[:10, 17:]

,stageType,country,kom,green,youth,time,DNF,DNS,OTL,DSQ,riderID
1,0,False,False,False,False,0 days 06:01:15.000000000,False,False,False,False,100
75,0,False,False,False,False,0 days 06:07:54.000000000,False,False,False,False,29
121,0,False,False,False,False,0 days 06:15:20.000000000,True,False,False,False,78
145,0,False,False,False,False,0 days 06:15:20.000000000,True,False,False,False,90
159,0,False,False,False,False,0 days 04:05:49.000000000,False,False,False,False,42
160,0,False,False,False,False,0 days 04:05:49.000000000,False,False,False,False,144
161,0,False,False,False,False,0 days 04:05:53.000000000,False,False,False,False,70
162,0,False,False,False,False,0 days 04:05:53.000000000,False,False,False,False,76
163,0,False,False,False,False,0 days 04:05:53.000000000,False,False,False,False,174
165,0,False,False,False,False,0 days 04:06:14.000000000,False,False,False,False,48


## Find test and training

In [70]:
stages_df = pd.read_csv(CLEANED_STAGES, index_col='stageID')
stages_df['dateFull'] = stages_df.dateFull.str.replace('th', '')
stages_df['dateFull'] = stages_df.dateFull.str.replace('rd', '')
stages_df['dateFull'] = stages_df.dateFull.str.replace('nd', '')

sst = stages_df[stages_df.dateFull.str.contains(r'(\d)+st')]
stages_df.loc[sst.index, 'dateFull'] = sst.dateFull.str.replace('st', '')
stages_df.loc[sst.index, 'dateFull'] = stages_df.dateFull.str.replace('Augu', 'August')
stages_df['dateFull'] = pd.to_datetime(stages_df['dateFull'])
stages_df.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,stage,url,distance,raceID,stageType,dateFull,averageSpeed,parcoursType,PCSPointScale,start,end,mountains,numMount,racePoints,scored
stageID,,,,,,,,,,,,,,,
0,Bretagne Classic - Ouest-France,race/bretagne-classic/2020/result,247.75,27,0,2020-08-25,41.15,45,1.WT.B,Plouay,Plouay,"['Restergal', 'Côte du Lezot', 'Restergal', 'C...",5,390,True
1,Cadel Evans Great Ocean Road Race,race/great-ocean-race/2020/result,171.10,28,0,2020-02-02,41.76,53,1.WT.C,Geelong,Geelong,[],0,395,True
2,Circuito de Getxo-Memorial Hermanos Otxoa,race/circuito-de-getxo/2020/result,177.00,29,0,2020-08-02,42.29,0,1.1,Getxo,Getxo,[],0,238,False
3,Clasica de Almeria,race/clasica-de-almeria/2020/result,187.60,30,0,2020-02-16,42.63,0,1.HC,Roquetas de Mar,Roquetas de Mar,[],0,397,False
4,European Continental Championships - ITT,race/uec-road-european-championships-itt/2020/...,25.60,31,6,2020-08-24,50.69,0,UCI.Cont.Ch.TT,Plouay,Plouay,[],0,86,False


In [64]:
limitDate = stages_df.loc[stages_df.url == 'race/tour-de-france/2020/stage-1', 'dateFull'].to_numpy()[0]
limitDate

numpy.datetime64('2020-08-29T00:00:00.000000000')

In [66]:
train_stages = stages_df[stages_df.dateFull < limitDate]
test_stages = stages_df[stages_df.dateFull >= limitDate]
print('{} training stages'.format(train_stages.shape[0]))
print('{} test stages'.format(test_stages.shape[0]))

181 training stages
12 test stages


In [69]:
train_race_history = races_hist[races_hist.stageID.isin(train_stages.index)]
test_race_history = races_hist[races_hist.stageID.isin(test_stages.index)]
print('{} training races history'.format(train_race_history.shape[0]))
print('{} test races_history'.format(test_race_history.shape[0]))

22876 training races history
2043 test races_history
